# Plot poloidal planes

In this tutorial we will learn how to plot FEEC data in poloidal planes of a toroidal geometry. In order for this notebook to run correctly, you need to first perform the following Struphy run:
```
    $ struphy run LinearMHD -i params_mhd.yml -o sim_mhd --mpi 2
```
and do the post processing via
```
    $ struphy pproc -d sim_mhd
```
After that, we can inspect the content of the `post_processing` folder (see the tutorial `diagnostics_1_standard_plots.ipynb`):

In [ ]:
import os
import struphy

path_out = os.path.join(struphy.__path__[0], 'io/out/', 'sim_mhd/')
data_path = os.path.join(path_out, 'post_processing')

os.listdir(data_path)

Since this was a run of the model `LinearMHD`, only `fields_data` is present in the output. Let us get the time grid and inspect the `fields_data`:

In [ ]:
import numpy as np

t_grid = np.load(os.path.join(data_path, 't_grid.npy'))
t_grid

In [ ]:
fluid_path = os.path.join(data_path, 'fields_data')

print(os.listdir(fluid_path))

The MHD variables (i.e. their values at predefined grid points) have been saved in the folder `mhd` (note that the magnetic field is stored under `em_fields`). But before we plot we want to have some more information on the geometry, initial conditions etc. For this, let us load the parameter file of the simulation:

In [ ]:
params_path = os.path.join(path_out, 'parameters.yml')

import yaml

with open(params_path) as file:
    parameters = yaml.load(file, Loader=yaml.FullLoader)

Let us look at the grid, geometry and fluid species:

In [ ]:
parameters['grid']

In [ ]:
parameters['geometry']

In [ ]:
parameters['fluid']

The simulation was done in `Tokamak` geometry with a resolution of `[8, 32, 4]`. In any toroidal geometry in Struphy, poloidal planes are always parametrized by the first two coordinates $(\eta_1, \eta_2)$ (here with a resolution `[8, 32]`), where $\eta_1$ parametrizes the radial coordinate and $\eta_2$ the angle coordinate. The toroidal angle is always parametrized by $\eta_3$ (here with a resolution of `[4]`). 

Moreover, we spot an MHD initial condition of type `TorusModesSin`, where the second (poloidal-angle-)component of the MHD velocity `uv` is initialized in logical coordinates with a poloidal mode number $m=3$, toroidal mode number $n=1$ and an amlitude 0.001. The radial profile is given by a half-period `sin` function (zero at the edges).

Let us first load the logical 1d grid vectors and the the MHD velocity `uv` with components in $(H^1(\hat\Omega))^3$. We then plot the initial condition of the logical velocity in the five available logical poloidal planes, which in Struphy are always spanned by $(\eta_1, \eta_2)$:

In [ ]:
import pickle

with open(os.path.join(fluid_path, 'grids_log.bin'), 'rb') as file:
    e1_grid, e2_grid, e3_grid = pickle.load(file)
    
print('Shape of the e1-grid:', e1_grid.shape)
print('Shape of the e2-grid:', e2_grid.shape)
print('Shape of the e3-grid:', e3_grid.shape)

n_poloidal_planes = e3_grid.size
    
with open(os.path.join(fluid_path, 'mhd/', 'uv_log.bin'), 'rb') as file:
    u_log = pickle.load(file)
    
u_log_init = u_log[0]
    
from matplotlib import pyplot as plt

plt.figure(figsize=(12, 16))
for n in range(n_poloidal_planes):
    plt.subplot(3, 2, n + 1)
    plt.contourf(e1_grid, e2_grid, u_log_init[1][:, :, n].T)
    plt.xlabel('$\eta_1$')
    plt.ylabel('$\eta_2$')
    plt.title(f'$\hat u_2(t=0)$ at $\eta_3={e3_grid[n]}$')
    plt.colorbar()

We can see the initial condition with a poloidal mode number $m=3$ (in the angle parameter $\eta_2$) and the half-sine function in the radial $\eta_1$ direction. The amplitude of 0.001 also corresponds to the value specified in the parameter file.

Let us now load the physical grid:

In [ ]:
with open(os.path.join(fluid_path, 'grids_phy.bin'), 'rb') as file:
    x_grid, y_grid, z_grid = pickle.load(file)
    
print('Shape of the x-grid:', x_grid.shape)
print('Shape of the y-grid:', y_grid.shape)
print('Shape of the z-grid:', z_grid.shape)

The physical grids are meshgrids.

It is a good exercise to compute the $R$- and $\phi$-grids from the Cartesian grid (please check out the [Struphy documentation](https://struphy.pages.mpcdf.de/struphy/sections/domains.html) for more details on the definition of the specific mappings). The mapping $(R, \phi, Z) \mapsto (x, y, z)$ is given by
$$
\begin{aligned}
 x &= R \cos(\phi) \\
 y &= -R\sin(\phi) \\
 z &= Z
 \end{aligned}
$$

We must pay attention to the parameter `tor_period` of the `Tokamak` geometry, which was set to 3, meaning that only one third of the total Torus has been simulated, $\phi \in (0, 2\pi/3)$. If we plot $3\phi_k$ for each $k = 0,...,4$ in the grid (i.e. looping over the third index), we should span the interval $(0, 2\pi)$:

In [ ]:
R_grid = np.sqrt(x_grid**2 + y_grid**2)
phi_grid = np.arctan2(-y_grid, x_grid)

for n in range(phi_grid.shape[2]):
    print(3 * phi_grid[0, 0, n])

Let us now plot the grids of the five available poloidal planes. Because Struphy uses tensor product grids, they must be identical:

In [ ]:
plt.figure(figsize=(12, 16))
for n in range(n_poloidal_planes):
    plt.subplot(3, 2, n + 1)
    # eta1-isolines
    for i in range(R_grid.shape[0]):
        plt.plot(R_grid[i, :, n], z_grid[i, :, n], 'tab:blue', alpha=.5, zorder=0)
    # eta1-isolines
    for j in range(R_grid.shape[1] - 1):
        plt.plot(R_grid[:, j, n], z_grid[:, j, n], 'tab:blue', alpha=.5, zorder=0)
    plt.xlabel('R')
    plt.ylabel('Z')
    plt.title(f'$\phi = {phi_grid[0, 0, n]}$')
    plt.axis('equal')

Let us now plot the second component of the MHD velocity $u \in (H^1(\Omega))^3$ on the physical grid, at $\phi = 0$:

In [ ]:
plt.figure(figsize=(12, 16))

for n, (t, u_comps) in enumerate(u_log.items()): 
    plt.subplot(3, 2, n + 1)
    plt.contourf(R_grid[:, :, 0], z_grid[:, :, 0], u_comps[1][:, :, 0])
    plt.colorbar()
    plt.title(f'$\hat u_2(R, 0, Z)$ at t={t}')
    plt.xlabel('R')
    plt.ylabel('Z')
    plt.axis('equal')

The velocity is here represented in the space $(H^1(\Omega))^3$, which in Struphy is the space for "vector fields" (=contra-variant components). We can also plot the co-variant components (1-form) of the velocity. For this, we need to recreate the domain used in the simulation and transform vector-field to 1-form components with  the `domain.transform` method: 

In [ ]:
from struphy.geometry.domains import Tokamak

params_map = parameters['geometry']['Tokamak']

domain = Tokamak(**params_map)

plt.figure(figsize=(12, 16))

for n, (t, u_comps) in enumerate(u_log.items()): 
    
    u1 = domain.transform(u_comps, e1_grid, e2_grid, e3_grid, kind='v_to_1')
    
    plt.subplot(3, 2, n + 1)
    plt.contourf(R_grid[:, :, 0], z_grid[:, :, 0], u1[1][:, :, 0])
    plt.colorbar()
    plt.title(f'1-form comp. $\hat u^1_2(R, 0, Z)$ at t={t}')
    plt.xlabel('R')
    plt.ylabel('Z')
    plt.axis('equal')